In [73]:
import pandas as pd 
import numpy as np
import mysql.connector

# MySQL Server에 연결
mydb = mysql.connector.connect()


In [63]:
import tensorflow as tf
from tensorflow import keras
from tensorflow import keras
from tensorflow.keras.layers import Dense, Flatten, SimpleRNN
from tensorflow.keras.layers import Input, LSTM, GRU
from tensorflow.keras.layers import Bidirectional, Conv1D, MaxPool1D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.backend import clear_session
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

In [45]:
aac_df = pd.read_csv('./aac_list(2).csv')

mycursor = mydb.cursor()
mycursor.execute("SELECT * FROM user_aac_history")

# 모든 결과를 소비
result = mycursor.fetchall()

# 데이터 프레임 생성
columns = [desc[0] for desc in mycursor.description]
aac_history_df = pd.DataFrame(result, columns=columns)

# 커서를 닫습니다.
mycursor.close()


True

In [46]:
def parse_aac_id(aac_id_str):
    return [int(num.strip('"')) for num in aac_id_str.split(',')]

# 'aac_id' 열 값을 숫자 배열로 변환
aac_history_df['aac_id'] = aac_history_df['aac_id'].apply(parse_aac_id)

In [47]:
aac_history_df.head()

,user_id,aac_id,id
0,1,"[1, 9, 5, 3, 1]",2
1,1,"[4, 6, 3, 1]",3
2,1,"[8, 1, 6, 3]",4
3,1,"[1, 2, 4, 7]",5
4,1,"[9, 6, 5]",6


In [49]:
temp = aac_history_df['aac_id'].tolist()
print(temp)
temp_ = []

for seq in temp :
    for i in range(1, len(seq)):
        temp_.append( seq[:i+1]    )

aac_seq = temp_
aac_seq[:20]

[[1, 9, 5, 3, 1], [4, 6, 3, 1], [8, 1, 6, 3], [1, 2, 4, 7], [9, 6, 5]]


[[1, 9],
 [1, 9, 5],
 [1, 9, 5, 3],
 [1, 9, 5, 3, 1],
 [4, 6],
 [4, 6, 3],
 [4, 6, 3, 1],
 [8, 1],
 [8, 1, 6],
 [8, 1, 6, 3],
 [1, 2],
 [1, 2, 4],
 [1, 2, 4, 7],
 [9, 6],
 [9, 6, 5]]

In [67]:
####################
## Your Code here ##
####################
from keras.preprocessing.sequence import pad_sequences #문장길이통일기

max_len = max( [ len(seq) for seq in aac_seq]      )
aac_seq = pad_sequences(aac_seq, maxlen=max_len, padding='pre')
aac_seq

array([[0, 0, 0, 1, 9],
       [0, 0, 1, 9, 5],
       [0, 1, 9, 5, 3],
       [1, 9, 5, 3, 1],
       [0, 0, 0, 4, 6],
       [0, 0, 4, 6, 3],
       [0, 4, 6, 3, 1],
       [0, 0, 0, 8, 1],
       [0, 0, 8, 1, 6],
       [0, 8, 1, 6, 3],
       [0, 0, 0, 1, 2],
       [0, 0, 1, 2, 4],
       [0, 1, 2, 4, 7],
       [0, 0, 0, 9, 6],
       [0, 0, 9, 6, 5]])

In [79]:
x = aac_seq[:, : -1] 
y = aac_seq[:, -1]
x = np.array(x)
x = x.reshape(x.shape[0], x.shape[1], 1)


In [80]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(x,y, test_size=0.3)
train_x, val_x, train_y, val_y  = train_test_split(train_x,train_y, test_size=0.1)

In [69]:
from keras.utils import to_categorical

# 나중에 total_aac 로 원핫인코딩
# y = to_categorical(y, num_classes = total_aac)

In [84]:
clear_session()
model = Sequential([
    LSTM(64, activation='relu', input_shape=(x.shape[1], x.shape[2])), # input_shape를 설정하여 입력 데이터의 형태에 맞춰줍니다.
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(1)  # 출력 값이 하나이므로, 출력 레이어에 한 개의 뉴런을 사용합니다.
])

# 모델 컴파일
model.compile(
    optimizer='adam',
    loss='mae',
    metrics='accuracy'
)

# 모델 훈련
history = model.fit(x, y, epochs=100, batch_size=32, verbose=0)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 64)                16896     
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 19521 (76.25 KB)
Trainable params: 19521 (76.25 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [85]:
## 학습도 시킬 것

lr_reduction = ReduceLROnPlateau(monitor='val_loss',
                                patience=4,
                                verbose=1,
                                factor=0.2,
                                min_lr = 0.000001)
es = EarlyStopping(monitor='val_loss',
                  min_delta = 0,
                  patience=5,
                  verbose=1,
                  restore_best_weights=True)

In [90]:

model.fit(train_x, train_y, epochs=30, verbose=1, validation_data=(val_x, val_y)
,callbacks=[lr_reduction, es],)

Epoch 1/30
1/1 [==============================] - 0s 235ms/step - loss: 1.0011 - accuracy: 0.1111 - val_loss: 2.9718 - val_accuracy: 0.0000e+00 - lr: 2.0000e-04
Epoch 2/30
1/1 [==============================] - 0s 65ms/step - loss: 1.0011 - accuracy: 0.1111 - val_loss: 2.9771 - val_accuracy: 0.0000e+00 - lr: 2.0000e-04
Epoch 3/30
1/1 [==============================] - 0s 96ms/step - loss: 0.9965 - accuracy: 0.1111 - val_loss: 2.9922 - val_accuracy: 0.0000e+00 - lr: 2.0000e-04
Epoch 4/30
1/1 [==============================] - 0s 87ms/step - loss: 0.9882 - accuracy: 0.1111 - val_loss: 3.0155 - val_accuracy: 0.0000e+00 - lr: 2.0000e-04
Epoch 5/30
1/1 [==============================] - ETA: 0s - loss: 0.9768 - accuracy: 0.1111
Epoch 5: ReduceLROnPlateau reducing learning rate to 4.0000001899898055e-05.
1/1 [==============================] - 0s 86ms/step - loss: 0.9768 - accuracy: 0.1111 - val_loss: 3.0457 - val_accuracy: 0.0000e+00 - lr: 2.0000e-04
Epoch 6/30
1/1 [=========================

In [94]:
pred_y = model.predict(test_x)
print(pred_y)
print(test_x)

1/1 [==============================] - 0s 38ms/step
[[4.242895 ]
 [4.3680563]
 [3.234794 ]
 [5.773211 ]
 [1.4502974]]
[[[0]
  [0]
  [0]
  [4]]

 [[0]
  [0]
  [4]
  [6]]

 [[0]
  [0]
  [0]
  [1]]

 [[0]
  [0]
  [0]
  [8]]

 [[1]
  [9]
  [5]
  [3]]]
